<a href="https://colab.research.google.com/github/dhan16/colabs/blob/master/covid19opendata/WikiData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [2]:

import requests
import pandas as pd

ENDPOINT = "https://query.wikidata.org/sparql"

def wikidata_to_dataframe(json):
  results = json["results"]["bindings"]
  columns=[]
  for result in results:
    if len(result) > len(columns):
      columns=[val for val in result]
  rows = []
  for result in results:
    values = [result[c]["value"] if c in result else None for c in columns ]
    rows.append(values)
  return pd.DataFrame(rows, columns=columns)

def wiki_data(sparql):
  res = requests.get(ENDPOINT, params = {'format': 'json', 'query': sparql})
  # print(res.json())
  return wikidata_to_dataframe(res.json())

In [3]:
!pip install --upgrade gspread

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


## Indonesia

### Wikidata

In [4]:
sparql = """
SELECT ?place ?subregion1Label ?placeLabel ?classLabel ?indonesia_admincode ?subregion1_endtime ?class_endtime
WHERE
{
  ?place wdt:P31/wdt:P279* wd:Q12479774. # P31=instance of, P279=subclass of
  ?place wdt:P131 ?subregion1. # P131=located in the administrative territorial entity
  OPTIONAL { ?place p:P131 [ps:P131 ?subregion1; pq:P582 ?subregion1_endtime ]. }
  ?place wdt:P2588 ?indonesia_admincode.
  ?place wdt:P31 ?class.
  OPTIONAL { ?place p:P31 [ps:P31 ?class; pq:P582 ?class_endtime ]. }
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
  }
}
"""
wiki_raw = wiki_data(sparql)
wiki_raw
len(wiki_raw)

679

In [5]:
def city_or_regency(classLabel: str) -> str:
  classLabel = classLabel.lower()
  if 'regency' in classLabel:
    return 'regency'
  elif 'city' in classLabel:
    return 'city'
  else:
    return 'Other'

wiki_df = wiki_raw.copy()
wiki_df = wiki_df[wiki_df.apply(lambda r : r.subregion1_endtime is None and r.class_endtime is None, axis=1)]
wiki_df['city_or_regency'] = wiki_df.apply(lambda r: city_or_regency(r.classLabel), axis=1)
wiki_df = wiki_df.drop(['classLabel'], axis=1)
wiki_df = wiki_df.drop_duplicates()

wiki_df
len(wiki_df)

528

In [6]:
wiki_df = wiki_df[wiki_df.subregion1Label != 'Q28725381']
duplicate = wiki_df[wiki_df.duplicated(['placeLabel', 'city_or_regency'], keep=False)] 
# len(duplicate)
duplicate


,place,class_endtime,indonesia_admincode,subregion1Label,placeLabel,subregion1_endtime,city_or_regency


### OpenCovid

In [7]:
meta = pd.read_csv('https://raw.githubusercontent.com/GoogleCloudPlatform/covid-19-open-data/main/src/data/metadata.csv')
meta = meta.query('(country_code == "ID")')
# meta

In [8]:
#  wiki_cities = wiki_cities.merge(meta, how='inner', left_on=['subregion1Label'], right_on=['subregion1_name']) 
#  wiki_regencies = wiki_regencies.merge(meta, how='inner', left_on=['subregion1Label'], right_on=['subregion1_name']) 
 

### Translations

In [9]:
def indonesian_direction_to_english(place: str):
  to_english = {
      'Pusat': 'Central',
      'Tengah' : 'Central',
      'Utara' : 'North',
      'Selatan' : 'South',
      'Timur' : 'East',
      'Barat' : 'West',
  }
  bits = place.split()
  if len(bits) > 1:
    for indo_dir, eng_dir in to_english.items():
      # in indonesian the dir is at the end 
      if bits[-1] == indo_dir:
        bits.pop()
        bits.insert(0, eng_dir)
        break
  place = ' '.join(bits)
  return place

def indonesian_islands_to_english(place: str):
  to_english = {
      'Kepulauan' : 'Islands',
      'Pulau' : 'Island',
  }
  bits = place.split()
  bits = [to_english.get(b, b) for b in bits]
  # in english, islands is at the end
  if len(bits) > 1:
    for eng in to_english.values():
      # in indonesian the dir is at the end 
      if bits[0] == eng:
        bits.pop(0)
        bits.append(eng)
        break
  place = ' '.join(bits)
  return place

[
  indonesian_direction_to_english('whatever Tengah'),
  indonesian_islands_to_english('Kepulauan Tengah')
]

['Central whatever', 'Tengah Islands']

In [10]:
spellings = {
    # cities
    'Bau-Bau' : 'Baubau',
    'Sungaipenuh' : 'Sungai Penuh',
    'Palangka Raya' : 'Palangkaraya',
    'Tidore Kepulauan' : 'Tidore', # https://www.wikidata.org/wiki/Q19153
    'Lubuk Linggau' : 'Lubuklinggau', # https://www.wikidata.org/wiki/Q8129
    'Pematang Siantar': 'Pematangsiantar', # https://www.wikidata.org/wiki/Q5979
    'Tanjung Balai' : 'Tanjungbalai', # https://www.wikidata.org/wiki/Q5987
    # regencies
    'Kupang' : 'Kupang Regency', # https://www.wikidata.org/wiki/Q14141
    'Toba Samosir' : 'Toba Regency', # https://www.wikidata.org/wiki/Q5911
    'Tojo Una-Una' : 'Tojo Una Una',
    'Toli-Toli': 'Tolitoli',
    'Muko Muko': 'Mukomuko', # https://www.wikidata.org/wiki/Q8033
    'Kepulauan Seribu' : 'Thousand Islands', # https://translate.google.com/?sl=id&tl=en&text=Kepulauan%20Seribu&op=translate
    'Penajam Paser Utara' : 'Penajam North Paser',
    'Pasangkayu (Mamuju Utara)': 'Pasangkayu', # https://en.wikipedia.org/wiki/Pasangkayu_Regency
    'Labuhanbatu' : 'Labuhan Batu', # https://www.wikidata.org/wiki/Q5814
    'Batu Bara': 'Batubara', # https://www.wikidata.org/wiki/Q5797
    'Limapuluh Kota': 'Lima Puluh Kota', # https://www.wikidata.org/wiki/Q6032
    'Batang Hari': 'Batanghari', # https://www.wikidata.org/wiki/Q7370
    'Kepulauan Sangihe': 'Sangihe', # https://www.wikidata.org/wiki/Q15839
    'Kepulauan Sitaro': 'Kepulauan Siau Tagulandang Biaro', # https://www.wikidata.org/wiki/Q15840
}
def translate_sheet_place_to_wikidata(place: str) -> str:
  if place in spellings:
    return spellings[place]
  place = indonesian_direction_to_english(place)
  place = indonesian_islands_to_english(place)
  return place

[
 translate_sheet_place_to_wikidata('Kepulauan Tengah'),
 translate_sheet_place_to_wikidata('Kepulauan Tenga'),
 translate_sheet_place_to_wikidata('Jakarta Barat'),
] 

def translate_wiki_subregion_to_metadata(place: str) -> str:
  translations = {
      'Special Region of Yogyakarta': 'Yogyakarta'
  }
  return translations.get(place, place)
 

### Sheet

In [11]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

sheet_url = 'https://docs.google.com/spreadsheets/d/1FJJXiGuOb5nXrjJeV3QcHNhTo38YdcsTIFl29mWDIqI/edit#gid=2006070746'
worksheet = gc.open_by_url(sheet_url).worksheet('Kode Kota')
rows = worksheet.get_all_values()
sheet_raw = pd.DataFrame.from_records(rows[2:], columns=rows[1])

In [12]:
def city_or_regency(KabKota: str) -> str:
  if KabKota == 'Kab.':
    return 'regency'
  elif KabKota == 'Kota':
    return 'city'
  else:
    return KabKota

sheet_df = sheet_raw.copy()
sheet_df = sheet_df[sheet_df.KabKota != 'zTam']
sheet_df['KabKota_eng'] = sheet_df.apply(lambda r: city_or_regency(r.KabKota), axis=1)
[set(sheet_df.KabKota_eng), len(sheet_df) ]

[{'city', 'regency'}, 514]

In [13]:
# Match rows in sheet with rows in wiki
sheet_df['Kota_translated'] = sheet_df.apply(lambda r: translate_sheet_place_to_wikidata(r.Kota), axis=1)
df = sheet_df.merge(wiki_df, how='left', left_on=['KabKota_eng', 'Kota'], right_on=['city_or_regency','placeLabel'], validate="1:1")

df1 = df[~df['placeLabel'].isnull()]
df2 = df[df['placeLabel'].isnull()]
df2 = df2[list(sheet_df.columns)]
df2 = df2.merge(wiki_df, how='left', left_on=['KabKota_eng', 'Kota_translated'], right_on=['city_or_regency','placeLabel'], validate="1:1")
df = pd.concat([df1, df2])

# df
# missing = df[df['placeLabel'].isnull()]
# missing
# len(missing)

In [14]:
# merge with meta
df['subregion1_translated'] = df.apply(lambda r: translate_wiki_subregion_to_metadata(r.subregion1Label), axis=1)
df = df.merge(meta, how='left', left_on=['subregion1_translated'], right_on=['subregion1_name'])
missing = df[df['subregion1_name'].isnull()]
missing
# len(missing)
# df


,ID Provinsi,Provinsi,ID Kota,KabKota,Kota,KabKota_eng,Kota_translated,place,class_endtime,indonesia_admincode,subregion1Label,placeLabel,subregion1_endtime,city_or_regency,subregion1_translated,key,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,locality_code,locality_name,match_string,aggregate_report_offset


In [15]:
df['country_code'] = 'ID'
df['country_name'] = 'Indonesia'
df['subregion2_code'] = df.apply(lambda r: r.indonesia_admincode.replace('.', ''), axis=1)
df['key'] = df.apply(lambda r: 'ID_' + r.subregion1_code + '_' + r.subregion2_code, axis=1)

df = df.sort_values(by=['key'])
cols = ['key', 'country_code', 'country_name', 'subregion1_code', 'subregion1_name', 'subregion2_code', 'placeLabel']

print(df[cols].to_csv(index=False))

key,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,placeLabel
ID_AC_1101,ID,Indonesia,AC,Aceh,1101,Aceh Selatan
ID_AC_1102,ID,Indonesia,AC,Aceh,1102,Aceh Tenggara
ID_AC_1103,ID,Indonesia,AC,Aceh,1103,Aceh Timur
ID_AC_1104,ID,Indonesia,AC,Aceh,1104,Aceh Tengah
ID_AC_1105,ID,Indonesia,AC,Aceh,1105,Aceh Barat
ID_AC_1106,ID,Indonesia,AC,Aceh,1106,Aceh Besar
ID_AC_1107,ID,Indonesia,AC,Aceh,1107,Pidie
ID_AC_1108,ID,Indonesia,AC,Aceh,1108,Aceh Utara
ID_AC_1109,ID,Indonesia,AC,Aceh,1109,Simeulue
ID_AC_1110,ID,Indonesia,AC,Aceh,1110,Aceh Singkil
ID_AC_1111,ID,Indonesia,AC,Aceh,1111,Bireuen
ID_AC_1112,ID,Indonesia,AC,Aceh,1112,Aceh Barat Daya
ID_AC_1113,ID,Indonesia,AC,Aceh,1113,Gayo Lues
ID_AC_1114,ID,Indonesia,AC,Aceh,1114,Aceh Jaya
ID_AC_1115,ID,Indonesia,AC,Aceh,1115,Nagan Raya
ID_AC_1116,ID,Indonesia,AC,Aceh,1116,Aceh Tamiang
ID_AC_1117,ID,Indonesia,AC,Aceh,1117,Bener Meriah
ID_AC_1118,ID,Indonesia,AC,Aceh,1118,Pidie Jaya
ID_AC_1171,ID,Indonesia,AC,Aceh,1171,Banda

In [16]:
len(df)

514

In [26]:
dictionary = dict(zip(list(df.subregion2_code), list(df['ID Kota'])))
for key, value in dictionary.items():
    print('  "' + key + '": ' + value + ",")

  "1101": 262,
  "1102": 266,
  "1103": 267,
  "1104": 265,
  "1105": 258,
  "1106": 260,
  "1107": 276,
  "1108": 268,
  "1109": 279,
  "1110": 263,
  "1111": 271,
  "1112": 259,
  "1113": 272,
  "1114": 261,
  "1115": 275,
  "1116": 264,
  "1117": 270,
  "1118": 277,
  "1171": 269,
  "1172": 278,
  "1173": 274,
  "1174": 273,
  "1175": 280,
  "5101": 6,
  "5102": 9,
  "5103": 1,
  "5104": 5,
  "5105": 8,
  "5106": 2,
  "5107": 7,
  "5108": 3,
  "5171": 4,
  "1901": 10,
  "1902": 14,
  "1903": 12,
  "1904": 13,
  "1905": 11,
  "1906": 15,
  "1971": 16,
  "1701": 26,
  "1702": 33,
  "1703": 28,
  "1704": 29,
  "1705": 34,
  "1706": 32,
  "1707": 31,
  "1708": 30,
  "1709": 27,
  "1771": 25,
  "3601": 19,
  "3602": 18,
  "3603": 22,
  "3604": 20,
  "3671": 23,
  "3672": 17,
  "3673": 21,
  "3674": 24,
  "7501": 48,
  "7502": 46,
  "7503": 47,
  "7504": 51,
  "7505": 50,
  "7571": 49,
  "1501": 55,
  "1502": 56,
  "1503": 58,
  "1504": 52,
  "1505": 57,
  "1506": 60,
  "1507": 61,
  "150